In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_inicio = "2025-01-01 00:00:00"
data_fim = "2025-06-01 00:00:00"
frequencia = "15T" # 15 minutos

print("Gerando dados sintéticos...")
datas = pd.date_range(start=data_inicio, end=data_fim, freq=frequencia)
df = pd.DataFrame(index=datas)

df['vazao'] = 0.2
perfil_diario = {
    0:0.1, 1:0.1, 2:0.1, 3:0.1, 4:0.2, 5:1.0,
    6:8.0, 7:15.0, 8:10.0, 9:5.0, 10:4.0, 11:7.0,
    12:12.0, 13:10.0, 14:4.0, 15:3.0, 16:3.0, 17:5.0,
    18:10.0, 19:18.0, 20:15.0, 21:8.0, 22:4.0, 23:1.0
}
perfil_semanal = {0:1.0, 1:1.0, 2:1.05, 3:1.0, 4:0.95, 5:1.2, 6:1.15}

horas = df.index.hour
dias_semana = df.index.dayofweek

df['vazao'] += (horas.map(perfil_diario) * dias_semana.map(perfil_semanal))
df['vazao'] += np.random.normal(0, 0.5, len(df)) # Ruído
df['vazao'] = df['vazao'].clip(lower=0) # Sem vazão negativa

df['volume_intervalo'] = df['vazao'] * 15.0


df['volume_acumulado_dia'] = df.groupby(df.index.date)['volume_intervalo'].cumsum()

df['vazao_futura'] = df['vazao'].shift(-1)


df = df.dropna()

df.to_csv('dados_vazao.csv')
print("Dataset gerado: 'dados_vazao_com_volume.csv'")
print(df[['vazao', 'volume_acumulado_dia', 'vazao_futura']].head())

Gerando dados sintéticos...


/tmp/ipython-input-2994136807.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  datas = pd.date_range(start=data_inicio, end=data_fim, freq=frequencia)


Dataset gerado: 'dados_vazao_com_volume.csv'
                        vazao  volume_acumulado_dia  vazao_futura
2025-01-01 00:00:00  0.000000              0.000000      0.554835
2025-01-01 00:15:00  0.554835              8.322531      0.008391
2025-01-01 00:30:00  0.008391              8.448402      0.000000
2025-01-01 00:45:00  0.000000              8.448402      0.000000
2025-01-01 01:00:00  0.000000              8.448402      0.000000


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('/content/dados_vazao.csv', index_col=0, parse_dates=True)

df['hora'] = df.index.hour
df['dia_semana'] = df.index.dayofweek

features = ['hora', 'dia_semana', 'vazao', 'volume_acumulado_dia']
target = 'vazao_futura'

X = df[features].values
y = df[target].values

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

print("\n--- COPIE ISTO PARA O C++ ---")
print(f"const float X_min[4] = {{ {', '.join([f'{x:.6f}' for x in scaler_X.data_min_])} }};")
print(f"const float X_scale[4] = {{ {', '.join([f'{x:.6f}' for x in scaler_X.scale_])} }};")
print(f"const float y_min = {scaler_y.data_min_[0]:.6f};")
print(f"const float y_scale = {scaler_y.scale_[0]:.6f};")
print("-----------------------------\n")


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=[4]), # 4 Entradas
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1) # 1 Saída (Previsão)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)


loss, mae = model.evaluate(X_test, y_test)
print(f"\nErro Médio Absoluto (Normalizado): {mae:.4f}")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('modelo_vazao_v2.tflite', 'wb') as f:
    f.write(tflite_model)

print("Modelo salvo: 'modelo_vazao.tflite'")
print("Não esqueça de rodar: xxd -i modelo_vazao_v2.tflite > modelo_vazao.h")


--- COPIE ISTO PARA O C++ ---
const float X_min[4] = { 0.000000, 0.000000, 0.000000, 0.000000 };
const float X_scale[4] = { 0.043478, 0.166667, 0.043631, 0.000093 };
const float y_min = 0.000000;
const float y_scale = 0.043631;
-----------------------------



/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0140 - mae: 0.0818
Epoch 2/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0084 - mae: 0.0567
Epoch 3/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0087 - mae: 0.0584
Epoch 4/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0082 - mae: 0.0564
Epoch 5/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0084 - mae: 0.0581
Epoch 6/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0081 - mae: 0.0571
Epoch 7/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0081 - mae: 0.0566
Epoch 8/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0082 - mae: 0.0583
Epoch 9/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0085 - mae: 0.0586
Epoch 10/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0082 - mae: 0.0579
Epoch 11/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0082 - mae: 0.0583
Epoch 12/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0081 - mae: 0.0581
Epoch 13/50
363/363 ━━━━━